In [230]:
#PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
import requests
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [231]:
# Define the API endpoint
api_url = "http://172.16.5.6:8080/v1/web/test12"

# Fetch data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a Python dictionary
    data = response.json()
    
    # Create a pandas DataFrame from the data
    # Assuming the API response is a list of dictionaries
    df = pd.DataFrame(data['data'])
    
else:
    print(f"Failed to fetch data: {response.status_code}")
    
data = df
print(data)

  branch agency partno                                                  D
0    999    99P   9901  [100, 105, 110, 108, 115, 120, 125, 130, 128, ...
1    999    99P   9902  [200, 210, 220, 215, 230, 240, 245, 250, 255, ...
2    999    99Q   9903  [50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...
3    888    88A   8804  [300, 310, 320, 315, 330, 340, 345, 350, 360, ...
4    888    88A   8805  [500, 505, 510, 515, 520, 525, 530, 535, 540, ...
5    999    99Q   9906  [150, 155, 160, 158, 165, 170, 175, 180, 185, ...
6    888    88B   8807  [400, 405, 410, 408, 415, 420, 425, 430, 435, ...
7    999    99Q   9908  [80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...
8    999    99R   9909  [250, 260, 270, 265, 280, 290, 295, 300, 310, ...
9    888    88B   8810  [600, 610, 620, 615, 630, 640, 645, 650, 660, ...


In [232]:
#file_path = "data/datadummy.json"
#with open(file_path,'r') as f:
    #json_data = json.load(f)
#data = pd.DataFrame(json_data)

#data = pd.DataFrame(json_data['data'])
#print(data)

In [233]:
#PARAMETER
#EWMA
alpha_ewma = 0.4

#SES & DES
alpha_ses = 0.65  # ubah nilai alpha (semakin besar semakin berat ke data terbaru)
beta_des = 0.45   # ubah nilai beta (semakin besar semakin cepat beradaptasi, kalo rendah bisa terjadi lag)

In [234]:
# Get mean and standard deviation of 12 periods before the last one
data['mean_12'] = data['D'].apply(lambda x: np.mean(x[-13:-1]))  # Use 12 periods before the last one
data['std_12'] = data['D'].apply(lambda x: np.std(x[-13:-1]))    # Use 12 periods before the last one

# Get upper bound from mean and std
data['ub'] = data['mean_12'] + 1.5 * data['std_12']

# Limit the original data to upper bound (using the 12 periods before the last one)
data['clipped_d'] = data.apply(lambda row: np.clip(row['D'][-13:-1], 0, row['ub']).tolist(), axis=1)

# Display the updated DataFrame
display(data)


,branch,agency,partno,D,mean_12,std_12,ub,clipped_d
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",135.083333,16.839727,160.342923,"[108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135..."
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",256.666667,21.245915,288.535539,"[215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260..."
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",84.416667,14.952471,106.845373,"[58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90...."
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",359.583333,23.314725,394.555420,"[315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365..."
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",543.333333,18.521759,571.115972,"[515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545..."
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",188.166667,18.783119,216.341345,"[158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190..."
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",438.166667,18.783119,466.341345,"[408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440..."
7,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",118.166667,18.783119,146.341345,"[88.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0..."
8,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",309.583333,23.314725,344.555420,"[265.0, 280.0, 290.0, 295.0, 300.0, 310.0, 315..."
9,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",659.583333,23.314725,694.555420,"[615.0, 630.0, 640.0, 645.0, 650.0, 660.0, 665..."


In [235]:
# Calculate Simple Moving Average
def calculate_ma(list):
    oldData = []
    newData = []
    for i in list:
        # store calculated data to old list
        oldData.append(i)
        newData.append(np.mean(oldData))
    return newData

data['ma'] = data['clipped_d'].apply(calculate_ma)
data['ma_result'] = data['ma'].apply(lambda x: x[-1:])
data['ma_result'] = data['clipped_d'].apply(lambda x: np.mean(x))
display(data)


,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",135.083333,16.839727,160.342923,"[108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135...","[108.0, 111.5, 114.33333333333333, 117.0, 119....",134.695244
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",256.666667,21.245915,288.535539,"[215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260...","[215.0, 222.5, 228.33333333333334, 232.5, 236....",256.544628
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",84.416667,14.952471,106.845373,"[58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90....","[58.0, 61.5, 64.33333333333333, 67.0, 69.6, 72...",84.153781
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",359.583333,23.314725,394.555420,"[315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365...","[315.0, 322.5, 328.3333333333333, 332.5, 336.0...",359.546285
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",543.333333,18.521759,571.115972,"[515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545...","[515.0, 517.5, 520.0, 522.5, 525.0, 527.5, 530...",543.009664
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",188.166667,18.783119,216.341345,"[158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190...","[158.0, 161.5, 164.33333333333334, 167.0, 169....",187.861779
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",438.166667,18.783119,466.341345,"[408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440...","[408.0, 411.5, 414.3333333333333, 417.0, 419.6...",437.861779
7,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",118.166667,18.783119,146.341345,"[88.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0...","[88.0, 91.5, 94.33333333333333, 97.0, 99.6, 10...",117.861779
8,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",309.583333,23.314725,344.555420,"[265.0, 280.0, 290.0, 295.0, 300.0, 310.0, 315...","[265.0, 272.5, 278.3333333333333, 282.5, 286.0...",309.546285
9,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",659.583333,23.314725,694.555420,"[615.0, 630.0, 640.0, 645.0, 650.0, 660.0, 665...","[615.0, 622.5, 628.3333333333334, 632.5, 636.0...",659.546285


In [236]:
import itertools
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd

data['wma_clipped_d'] = data.apply(lambda row: np.clip(row['D'][-15:], 0, row['ub']).tolist(), axis=1)

def wma_forecast_with_weights(data, weights):
    wma_values = [None] * 3
    for i in range(3, len(data)):
        forecast = np.sum(np.array(data[i-3:i]) * weights) / sum(weights)
        wma_values.append(forecast)
    return wma_values

def generate_weights(step=0.05):
    weights = []
    for w1 in np.arange(0.01, 1, step):
        for w2 in np.arange(w1 + 0.01, 1 - w1, step):
            w3 = 1 - w1 - w2
            if w3 > w2 > w1 > 0 and abs(w1 + w2 + w3 - 1) < 1e-6:
                weights.append((w1, w2, w3))
    return weights

best_weights_list = []
best_maes = []

for row in data['wma_clipped_d']:
    best_mae = float('inf')
    best_weights = None
    for weights in generate_weights(step=0.05):
        wma_values = wma_forecast_with_weights(row, weights)
        mae = mean_absolute_error(row[-12:], wma_values[-12:])
        if mae < best_mae:
            best_mae = mae
            best_weights = weights
    best_weights_list.append(best_weights)
    best_maes.append(best_mae)

data['best_weights'] = best_weights_list
data['best_mae'] = best_maes

data['wma'], data['wma_result'] = zip(*data.apply(lambda row: (
    wma_forecast_with_weights(row['wma_clipped_d'], row['best_weights'])[3:][-12:],
    wma_forecast_with_weights(row['wma_clipped_d'], row['best_weights'])[-12:]
), axis=1))

display(data)


,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result,wma_clipped_d,best_weights,best_mae,wma,wma_result
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",135.083333,16.839727,160.342923,"[108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135...","[108.0, 111.5, 114.33333333333333, 117.0, 119....",134.695244,"[105.0, 110.0, 108.0, 115.0, 120.0, 125.0, 130...","(0.01, 0.02, 0.97)",4.833601,"[108.00999999999999, 114.81, 119.7799999999999...","[108.00999999999999, 114.81, 119.7799999999999..."
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",256.666667,21.245915,288.535539,"[215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260...","[215.0, 222.5, 228.33333333333334, 232.5, 236....",256.544628,"[210.0, 220.0, 215.0, 230.0, 240.0, 245.0, 250...","(0.01, 0.02, 0.97)",6.361800,"[215.04999999999998, 229.6, 239.54999999999998...","[215.04999999999998, 229.6, 239.54999999999998..."
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",84.416667,14.952471,106.845373,"[58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90....","[58.0, 61.5, 64.33333333333333, 67.0, 69.6, 72...",84.153781,"[55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80.0, 85....","(0.01, 0.02, 0.97)",4.229228,"[58.01, 64.81, 69.77999999999999, 74.8, 79.8, ...","[58.01, 64.81, 69.77999999999999, 74.8, 79.8, ..."
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",359.583333,23.314725,394.555420,"[315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365...","[315.0, 322.5, 328.3333333333333, 332.5, 336.0...",359.546285,"[310.0, 320.0, 315.0, 330.0, 340.0, 345.0, 350...","(0.01, 0.02, 0.97)",6.882674,"[315.05, 329.59999999999997, 339.55, 344.75, 3...","[315.05, 329.59999999999997, 339.55, 344.75, 3..."
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",543.333333,18.521759,571.115972,"[515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545...","[515.0, 517.5, 520.0, 522.5, 525.0, 527.5, 530...",543.009664,"[505.0, 510.0, 515.0, 520.0, 525.0, 530.0, 535...","(0.01, 0.02, 0.97)",4.883288,"[514.8, 519.8, 524.8, 529.8000000000001, 534.8...","[514.8, 519.8, 524.8, 529.8000000000001, 534.8..."
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",188.166667,18.783119,216.341345,"[158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190...","[158.0, 161.5, 164.33333333333334, 167.0, 169....",187.861779,"[155.0, 160.0, 158.0, 165.0, 170.0, 175.0, 180...","(0.01, 0.02, 0.97)",5.052632,"[158.01, 164.80999999999997, 169.78, 174.8, 17...","[158.01, 164.80999999999997, 169.78, 174.8, 17..."
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",438.166667,18.783119,466.341345,"[408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440...","[408.0, 411.5, 414.3333333333333, 417.0, 419.6...",437.861779,"[405.0, 410.0, 408.0, 415.0, 420.0, 425.0, 430...","(0.01, 0.02, 0.97)",5.052632,"[408.01, 414.81, 419.78, 424.8, 429.7999999999...","[408.01, 414.81, 419.78, 424.8, 429.7999999999..."
7,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",118.166667,18.783119,146.341345,"[88.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0...","[88.0, 91.5, 94.33333333333333, 97.0, 99.6, 10...",117.861779,"[85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 110.0, ...","(0.01, 0.02, 0.97)",5.052632,"[88.01, 94.80999999999999, 99.78, 104.8, 109.8...","[88.01, 94.80999999999999, 99.78, 104.8, 109.8..."
8,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",309.583333,23.314725,344.555420,"[265.0, 280.0, 290.0, 295.0, 300.0, 310.0, 315...","[265.0, 272.5, 278.3333333333333, 282.5, 286.0...",309.546285,"[260.0, 270.0, 265.0, 280.0, 290.0, 295.0, 300...","(0.01, 0.02, 0.97)",6.882674,"[265.05, 279.59999999999997, 289.55, 294.75, 2...","[265.05, 279.59999999999997, 289.55, 294.75, 2..."
9,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",659.583333,23.314725,694.555420,"[615.0, 630.0, 640.0, 645.0, 650.0, 660.0, 665...","[615.0, 622.5, 628.3333333333334, 632.5, 636.0...",659.546285,"[610.0, 620.0, 615.0, 630.0, 640.0, 645.0, 650...","(0.01, 0.02, 0.97)"

In [237]:
# Calculate Exponential Weighted Moving Average (EWMA)
def ewma(list, alpha = alpha_ewma):
    df = pd.DataFrame(list)
    df['ewma'] = df.ewm(alpha=alpha_ewma, adjust=False).mean()
    return df['ewma'].tolist()

def ewma_forecast(list, alpha):
    ewma_values = ewma(list, alpha)
    if len(ewma_values) > 0:
        # Prediction for the next period
        next_forecast = (1 - alpha) * ewma_values[-1]
    else:
        next_forecast = None
    return ewma_values, next_forecast

data['ewma'], data['ewma_result'] = zip(*data['clipped_d'].apply(lambda x: ewma_forecast(x, alpha_ewma)))

display(data)

,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result,wma_clipped_d,best_weights,best_mae,wma,wma_result,ewma,ewma_result
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",135.083333,16.839727,160.342923,"[108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135...","[108.0, 111.5, 114.33333333333333, 117.0, 119....",134.695244,"[105.0, 110.0, 108.0, 115.0, 120.0, 125.0, 130...","(0.01, 0.02, 0.97)",4.833601,"[108.00999999999999, 114.81, 119.7799999999999...","[108.00999999999999, 114.81, 119.7799999999999...","[108.0, 110.8, 114.47999999999999, 118.6879999...",92.375860
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",256.666667,21.245915,288.535539,"[215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260...","[215.0, 222.5, 228.33333333333334, 232.5, 236....",256.544628,"[210.0, 220.0, 215.0, 230.0, 240.0, 245.0, 250...","(0.01, 0.02, 0.97)",6.361800,"[215.04999999999998, 229.6, 239.54999999999998...","[215.04999999999998, 229.6, 239.54999999999998...","[215.0, 221.0, 228.6, 235.16, 241.096, 246.657...",168.044947
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",84.416667,14.952471,106.845373,"[58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90....","[58.0, 61.5, 64.33333333333333, 67.0, 69.6, 72...",84.153781,"[55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80.0, 85....","(0.01, 0.02, 0.97)",4.229228,"[58.01, 64.81, 69.77999999999999, 74.8, 79.8, ...","[58.01, 64.81, 69.77999999999999, 74.8, 79.8, ...","[58.0, 60.8, 64.47999999999999, 68.68799999999...",60.363297
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",359.583333,23.314725,394.555420,"[315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365...","[315.0, 322.5, 328.3333333333333, 332.5, 336.0...",359.546285,"[310.0, 320.0, 315.0, 330.0, 340.0, 345.0, 350...","(0.01, 0.02, 0.97)",6.882674,"[315.05, 329.59999999999997, 339.55, 344.75, 3...","[315.05, 329.59999999999997, 339.55, 344.75, 3...","[315.0, 321.0, 328.6, 335.15999999999997, 341....",231.205738
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",543.333333,18.521759,571.115972,"[515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545...","[515.0, 517.5, 520.0, 522.5, 525.0, 527.5, 530...",543.009664,"[505.0, 510.0, 515.0, 520.0, 525.0, 530.0, 535...","(0.01, 0.02, 0.97)",4.883288,"[514.8, 519.8, 524.8, 529.8000000000001, 534.8...","[514.8, 519.8, 524.8, 529.8000000000001, 534.8...","[515.0, 517.0, 520.2, 524.12, 528.472, 533.083...",338.504159
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",188.166667,18.783119,216.341345,"[158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190...","[158.0, 161.5, 164.33333333333334, 167.0, 169....",187.861779,"[155.0, 160.0, 158.0, 165.0, 170.0, 175.0, 180...","(0.01, 0.02, 0.97)",5.052632,"[158.01, 164.80999999999997, 169.78, 174.8, 17...","[158.01, 164.80999999999997, 169.78, 174.8, 17...","[158.0, 160.8, 164.48000000000002, 168.688, 17...",125.553895
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",438.166667,18.783119,466.341345,"[408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440...","[408.0, 411.5, 414.3333333333333, 417.0, 419.6...",437.861779,"[405.0, 410.0, 408.0, 415.0, 420.0, 425.0, 430...","(0.01, 0.02, 0.97)",5.052632,"[408.01, 414.81, 419.78, 424.8, 429.7999999999...","[408.01, 414.81, 419.78, 424.8, 429.7999999999...","[408.0, 410.79999999999995, 414.47999999999996...",275.553895
7,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",118.166667,18.783119,146.341345,"[88.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0...","[88.0, 91.5, 94.33333333333333, 97.0, 99.6, 10...",117.861779,"[85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 110.0, ...","(0.01, 0.02, 0.97)",5.052632,"[88.01, 94.80999999999999, 99.78, 104.8, 109.8...","[88.01, 94.80999999999999, 99.78, 104.8, 109.8...","[88.0, 90.8, 94.47999999999999, 98.68799999999...",83.553895
8,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",309.583333,23.314725,344.555420,"[265.0, 280.0, 290.0, 295.0, 300.0, 310.0, 315...","[265.0, 272.5, 2

In [238]:
#LINEAR REGRESSION
#  Calculate Linear Regression
def lr(x):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    model =  LinearRegression()
    model.fit(df[['x']], df['y'])
    df.loc[len(df), 'x'] = len(df) + 1
    return model.predict(df[['x']])

data['lr'] = data['clipped_d'].apply(lambda x: lr(x))
data['lr_result'] = data['lr'].apply(lambda x: x[-1:])
display(data)


,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result,wma_clipped_d,best_weights,best_mae,wma,wma_result,ewma,ewma_result,lr,lr_result
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",135.083333,16.839727,160.342923,"[108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135...","[108.0, 111.5, 114.33333333333333, 117.0, 119....",134.695244,"[105.0, 110.0, 108.0, 115.0, 120.0, 125.0, 130...","(0.01, 0.02, 0.97)",4.833601,"[108.00999999999999, 114.81, 119.7799999999999...","[108.00999999999999, 114.81, 119.7799999999999...","[108.0, 110.8, 114.47999999999999, 118.6879999...",92.375860,"[109.16116370558575, 113.80372368481912, 118.4...",[164.87188345638617]
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",256.666667,21.245915,288.535539,"[215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260...","[215.0, 222.5, 228.33333333333334, 232.5, 236....",256.544628,"[210.0, 220.0, 215.0, 230.0, 240.0, 245.0, 250...","(0.01, 0.02, 0.97)",6.361800,"[215.04999999999998, 229.6, 239.54999999999998...","[215.04999999999998, 229.6, 239.54999999999998...","[215.0, 221.0, 228.6, 235.16, 241.096, 246.657...",168.044947,"[223.39287966325364, 229.42047030973208, 235.4...",[295.72396742099505]
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",84.416667,14.952471,106.845373,"[58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90....","[58.0, 61.5, 64.33333333333333, 67.0, 69.6, 72...",84.153781,"[55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80.0, 85....","(0.01, 0.02, 0.97)",4.229228,"[58.01, 64.81, 69.77999999999999, 74.8, 79.8, ...","[58.01, 64.81, 69.77999999999999, 74.8, 79.8, ...","[58.0, 60.8, 64.47999999999999, 68.68799999999...",60.363297,"[61.26341369733328, 65.42529867904238, 69.5871...",[111.20603347784257]
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",359.583333,23.314725,394.555420,"[315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365...","[315.0, 322.5, 328.3333333333333, 332.5, 336.0...",359.546285,"[310.0, 320.0, 315.0, 330.0, 340.0, 345.0, 350...","(0.01, 0.02, 0.97)",6.882674,"[315.05, 329.59999999999997, 339.55, 344.75, 3...","[315.05, 329.59999999999997, 339.55, 344.75, 3...","[315.0, 321.0, 328.6, 335.15999999999997, 341....",231.205738,"[322.8134076388234, 329.49211261990365, 336.17...",[402.9578674117865]
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",543.333333,18.521759,571.115972,"[515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545...","[515.0, 517.5, 520.0, 522.5, 525.0, 527.5, 530...",543.009664,"[505.0, 510.0, 515.0, 520.0, 525.0, 530.0, 535...","(0.01, 0.02, 0.97)",4.883288,"[514.8, 519.8, 524.8, 529.8000000000001, 534.8...","[514.8, 519.8, 524.8, 529.8000000000001, 534.8...","[515.0, 517.0, 520.2, 524.12, 528.472, 533.083...",338.504159,"[514.4082087439808, 519.6084733935139, 524.808...",[576.8113845383771]
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",188.166667,18.783119,216.341345,"[158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190...","[158.0, 161.5, 164.33333333333334, 167.0, 169....",187.861779,"[155.0, 160.0, 158.0, 165.0, 170.0, 175.0, 180...","(0.01, 0.02, 0.97)",5.052632,"[158.01, 164.80999999999997, 169.78, 174.8, 17...","[158.01, 164.80999999999997, 169.78, 174.8, 17...","[158.0, 160.8, 164.48000000000002, 168.688, 17...",125.553895,"[158.78957115199597, 164.07542707912447, 169.3...",[222.2198422775377]
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",438.166667,18.783119,466.341345,"[408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440...","[408.0, 411.5, 414.3333333333333, 417.0, 419.6...",437.861779,"[405.0, 410.0, 408.0, 415.0, 420.0, 425.0, 430...","(0.01, 0.02, 0.97)",5.052632,"[408.01, 414.81, 419.78, 424.8, 429.7999999999...","[408.01, 414.81, 419.78, 424.8, 429.7999999999...","[408.0, 410.79999999999995, 414.47999999999996...",275.553895,"[408.789571151996, 414.07542707912444, 419.361...",[472.21984227753774]
7,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",118.166667,18.783119,146.341345,"[88.0, 95

In [239]:
#POLYNOMIAL 2ND AND 3RD
# Calculate Polynomial Regression
def pr(x, pr_degree):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)

    X = df[['x']]  # Independent variable (reshape to 2D array)
    y = df['y']    # Dependent variable

    poly = PolynomialFeatures(degree=pr_degree)  # Create polynomial features
    X_poly = poly.fit_transform(X)  # Transform input features
    poly_model = LinearRegression()  # Initialize linear regression model
    poly_model.fit(X_poly, y)  # Fit polynomial model

    df.loc[len(df), 'x'] = len(df) + 1
    X_all_poly = poly.transform(df[['x']])
    return poly_model.predict(X_all_poly)  

data['pr2'] = data['clipped_d'].apply(lambda x: pr(x, 2))
data['pr2_result'] = data['pr2'].apply(lambda x: x[-1:])
data['pr3'] = data['clipped_d'].apply(lambda x: pr(x, 3))
data['pr3_result'] = data['pr3'].apply(lambda x: x[-1:])
display(data)


,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result,...,wma,wma_result,ewma,ewma_result,lr,lr_result,pr2,pr2_result,pr3,pr3_result
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",135.083333,16.839727,160.342923,"[108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135...","[108.0, 111.5, 114.33333333333333, 117.0, 119....",134.695244,...,"[108.00999999999999, 114.81, 119.7799999999999...","[108.00999999999999, 114.81, 119.7799999999999...","[108.0, 110.8, 114.47999999999999, 118.6879999...",92.375860,"[109.16116370558575, 113.80372368481912, 118.4...",[164.87188345638617],"[110.43799873444893, 114.38410324339328, 118.4...",[166.9844650495961],"[109.00648295122954, 114.51424104186773, 119.3...",[170.93197826998875]
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",256.666667,21.245915,288.535539,"[215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260...","[215.0, 222.5, 228.33333333333334, 232.5, 236....",256.544628,...,"[215.04999999999998, 229.6, 239.54999999999998...","[215.04999999999998, 229.6, 239.54999999999998...","[215.0, 221.0, 228.6, 235.16, 241.096, 246.657...",168.044947,"[223.39287966325364, 229.42047030973208, 235.4...",[295.72396742099505],"[220.5057671379066, 228.10814643457437, 235.39...",[290.9471085154206],"[216.77709550535175, 228.44711658298786, 237.7...",[301.22920301731403]
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",84.416667,14.952471,106.845373,"[58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90....","[58.0, 61.5, 64.33333333333333, 67.0, 69.6, 72...",84.153781,...,"[58.01, 64.81, 69.77999999999999, 74.8, 79.8, ...","[58.01, 64.81, 69.77999999999999, 74.8, 79.8, ...","[58.0, 60.8, 64.47999999999999, 68.68799999999...",60.363297,"[61.26341369733328, 65.42529867904238, 69.5871...",[111.20603347784257],"[59.695169758890174, 64.7124605252046, 69.5586...",[108.61130259787308],"[57.57403312872045, 64.90529112794655, 70.9084...",[114.46049754773546]
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",359.583333,23.314725,394.555420,"[315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365...","[315.0, 322.5, 328.3333333333333, 332.5, 336.0...",359.546285,...,"[315.05, 329.59999999999997, 339.55, 344.75, 3...","[315.05, 329.59999999999997, 339.55, 344.75, 3...","[315.0, 321.0, 328.6, 335.15999999999997, 341....",231.205738,"[322.8134076388234, 329.49211261990365, 336.17...",[402.9578674117865],"[319.38185142520064, 327.93231434098425, 336.1...",[397.2802016765196],"[316.5592304395168, 328.1889162487747, 337.904...",[405.0637928794649]
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",543.333333,18.521759,571.115972,"[515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545...","[515.0, 517.5, 520.0, 522.5, 525.0, 527.5, 530...",543.009664,...,"[514.8, 519.8, 524.8, 529.8000000000001, 534.8...","[514.8, 519.8, 524.8, 529.8000000000001, 534.8...","[515.0, 517.0, 520.2, 524.12, 528.472, 533.083...",338.504159,"[514.4082087439808, 519.6084733935139, 524.808...",[576.8113845383771],"[515.2615899199526, 519.9963739280465, 524.824...",[578.2233424840756],"[515.12167280906, 520.0090936654009, 524.91329...",[578.6091745171427]
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",188.166667,18.783119,216.341345,"[158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190...","[158.0, 161.5, 164.33333333333334, 167.0, 169....",187.861779,...,"[158.01, 164.80999999999997, 169.78, 174.8, 17...","[158.01, 164.80999999999997, 169.78, 174.8, 17...","[158.0, 160.8, 164.48000000000002, 168.688, 17...",125.553895,"[158.78957115199597, 164.07542707912447, 169.3...",[222.2198422775377],"[159.19604540013574, 164.26018810100618, 169.3...",[222.89237239718705],"[158.58537626246803, 164.31570347715817, 169.7...",[224.57633880711876]
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",438.166667,18.783119,466.341345,"[408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440...","[408.0, 411.5, 414.3333333333333, 417.0, 419.6...",437.861779,...,"[408.01, 414.81, 419.78, 4

In [240]:
#SES
def ses(x, alpha = alpha_ses):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    df.loc[len(df), 'x'] = len(df) + 1

    new_data = SimpleExpSmoothing(df['y']).fit(smoothing_level=alpha, optimized=False).fittedvalues
    return new_data.tolist()

data['ses'] = data['clipped_d'].apply(lambda x: ses(x, alpha_ses))
data['ses_result'] = data['ses'].apply(lambda x: x[-1:])
display(data)

,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result,...,ewma,ewma_result,lr,lr_result,pr2,pr2_result,pr3,pr3_result,ses,ses_result
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",135.083333,16.839727,160.342923,"[108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135...","[108.0, 111.5, 114.33333333333333, 117.0, 119....",134.695244,...,"[108.0, 110.8, 114.47999999999999, 118.6879999...",92.375860,"[109.16116370558575, 113.80372368481912, 118.4...",[164.87188345638617],"[110.43799873444893, 114.38410324339328, 118.4...",[166.9844650495961],"[109.00648295122954, 114.51424104186773, 119.3...",[170.93197826998875],"[108.0, 108.0, 112.55, 117.3925, 122.337375, 1...",[158.6689262240763]
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",256.666667,21.245915,288.535539,"[215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260...","[215.0, 222.5, 228.33333333333334, 232.5, 236....",256.544628,...,"[215.0, 221.0, 228.6, 235.16, 241.096, 246.657...",168.044947,"[223.39287966325364, 229.42047030973208, 235.4...",[295.72396742099505],"[220.5057671379066, 228.10814643457437, 235.39...",[290.9471085154206],"[216.77709550535175, 228.44711658298786, 237.7...",[301.22920301731403],"[215.0, 215.0, 224.75, 234.6625, 241.381874999...",[285.7430839324752]
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",84.416667,14.952471,106.845373,"[58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90....","[58.0, 61.5, 64.33333333333333, 67.0, 69.6, 72...",84.153781,...,"[58.0, 60.8, 64.47999999999999, 68.68799999999...",60.363297,"[61.26341369733328, 65.42529867904238, 69.5871...",[111.20603347784257],"[59.695169758890174, 64.7124605252046, 69.5586...",[108.61130259787308],"[57.57403312872045, 64.90529112794655, 70.9084...",[114.46049754773546],"[58.0, 58.0, 62.55, 67.3925, 72.337375, 77.318...",[104.79489339013506]
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",359.583333,23.314725,394.555420,"[315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365...","[315.0, 322.5, 328.3333333333333, 332.5, 336.0...",359.546285,...,"[315.0, 321.0, 328.6, 335.15999999999997, 341....",231.205738,"[322.8134076388234, 329.49211261990365, 336.17...",[402.9578674117865],"[319.38185142520064, 327.93231434098425, 336.1...",[397.2802016765196],"[316.5592304395168, 328.1889162487747, 337.904...",[405.0637928794649],"[315.0, 315.0, 324.75, 334.6625, 341.381875, 3...",[391.4027901314929]
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",543.333333,18.521759,571.115972,"[515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545...","[515.0, 517.5, 520.0, 522.5, 525.0, 527.5, 530...",543.009664,...,"[515.0, 517.0, 520.2, 524.12, 528.472, 533.083...",338.504159,"[514.4082087439808, 519.6084733935139, 524.808...",[576.8113845383771],"[515.2615899199526, 519.9963739280465, 524.824...",[578.2233424840756],"[515.12167280906, 520.0090936654009, 524.91329...",[578.6091745171427],"[515.0, 515.0, 518.25, 522.6375, 527.423125, 5...",[569.1705999697133]
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",188.166667,18.783119,216.341345,"[158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190...","[158.0, 161.5, 164.33333333333334, 167.0, 169....",187.861779,...,"[158.0, 160.8, 164.48000000000002, 168.688, 17...",125.553895,"[158.78957115199597, 164.07542707912447, 169.3...",[222.2198422775377],"[159.19604540013574, 164.26018810100618, 169.3...",[222.89237239718705],"[158.58537626246803, 164.31570347715817, 169.7...",[224.57633880711876],"[158.0, 158.0, 162.55, 167.39249999999998, 172...",[214.31707325124512]
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",438.166667,18.783119,466.341345,"[408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440...","[408.0, 411.5, 414.3333333333333, 417.0, 419.6...",437.861779,...,"[408.0, 410.79999999999995, 414.47999999999996...",275.553895,"[408.789571151996, 414.07542707912444, 419.361...",[472.21984227753774],"[409.1960454001358, 414.2601881010062, 419.368...",[472.8923723971871],"[408.585

In [241]:
#DES
def des(x, alpha = alpha_ses, beta = beta_des):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    df.loc[len(df), 'x'] = len(df) + 1

    new_data = ExponentialSmoothing(df['y'], trend='add', seasonal=None).fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False).fittedvalues
    return new_data.tolist()

data['des'] = data['clipped_d'].apply(lambda x: des(x,alpha_ses, beta_des))
data['des_result'] = data['des'].apply(lambda x: x[-1:])
display(data)

,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result,...,lr,lr_result,pr2,pr2_result,pr3,pr3_result,ses,ses_result,des,des_result
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",135.083333,16.839727,160.342923,"[108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135...","[108.0, 111.5, 114.33333333333333, 117.0, 119....",134.695244,...,"[109.16116370558575, 113.80372368481912, 118.4...",[164.87188345638617],"[110.43799873444893, 114.38410324339328, 118.4...",[166.9844650495961],"[109.00648295122954, 114.51424104186773, 119.3...",[170.93197826998875],"[108.0, 108.0, 112.55, 117.3925, 122.337375, 1...",[158.6689262240763],"[110.07272727272726, 112.45857575757576, 118.5...",[167.0467857148429]
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",256.666667,21.245915,288.535539,"[215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260...","[215.0, 222.5, 228.33333333333334, 232.5, 236....",256.544628,...,"[223.39287966325364, 229.42047030973208, 235.4...",[295.72396742099505],"[220.5057671379066, 228.10814643457437, 235.39...",[290.9471085154206],"[216.77709550535175, 228.44711658298786, 237.7...",[301.22920301731403],"[215.0, 215.0, 224.75, 234.6625, 241.381874999...",[285.7430839324752],"[223.36363636363626, 221.5112121212121, 233.09...",[294.9825658666633]
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",84.416667,14.952471,106.845373,"[58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90....","[58.0, 61.5, 64.33333333333333, 67.0, 69.6, 72...",84.153781,...,"[61.26341369733328, 65.42529867904238, 69.5871...",[111.20603347784257],"[59.695169758890174, 64.7124605252046, 69.5586...",[108.61130259787308],"[57.57403312872045, 64.90529112794655, 70.9084...",[114.46049754773546],"[58.0, 58.0, 62.55, 67.3925, 72.337375, 77.318...",[104.79489339013506],"[61.63636363636361, 62.24545454545455, 67.8143...",[111.53189810212976]
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",359.583333,23.314725,394.555420,"[315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365...","[315.0, 322.5, 328.3333333333333, 332.5, 336.0...",359.546285,...,"[322.8134076388234, 329.49211261990365, 336.17...",[402.9578674117865],"[319.38185142520064, 327.93231434098425, 336.1...",[397.2802016765196],"[316.5592304395168, 328.1889162487747, 337.904...",[405.0637928794649],"[315.0, 315.0, 324.75, 334.6625, 341.381875, 3...",[391.4027901314929],"[322.4545454545453, 322.2165151515152, 334.159...",[401.0374880961348]
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",543.333333,18.521759,571.115972,"[515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545...","[515.0, 517.5, 520.0, 522.5, 525.0, 527.5, 530...",543.009664,...,"[514.4082087439808, 519.6084733935139, 524.808...",[576.8113845383771],"[515.2615899199526, 519.9963739280465, 524.824...",[578.2233424840756],"[515.12167280906, 520.0090936654009, 524.91329...",[578.6091745171427],"[515.0, 515.0, 518.25, 522.6375, 527.423125, 5...",[569.1705999697133],"[514.9999999999997, 520.0, 525.0000000000001, ...",[577.7852721686249]
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",188.166667,18.783119,216.341345,"[158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190...","[158.0, 161.5, 164.33333333333334, 167.0, 169....",187.861779,...,"[158.78957115199597, 164.07542707912447, 169.3...",[222.2198422775377],"[159.19604540013574, 164.26018810100618, 169.3...",[222.89237239718705],"[158.58537626246803, 164.31570347715817, 169.7...",[224.57633880711876],"[158.0, 158.0, 162.55, 167.39249999999998, 172...",[214.31707325124512],"[159.30909090909086, 163.18436363636366, 169.6...",[222.98920400819821]
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",438.166667,18.783119,466.341345,"[408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440...","[408.0, 411.5, 414.3333333333333, 417.0, 419.6...",437.861779,...,"[408.789571151996, 414.07542707912444, 419.361...",[472.21984227753774],"[409.1960454001358, 414.2601881010062, 419.368...",[472.8923723971871],"[408.5853762624

In [242]:
def metric(x):
    period_length = len(x['clipped_d'])
    df = pd.DataFrame()
    df['period'] = range(1, period_length + 1)
    df['qty'] = x['clipped_d'][:period_length]  # Ground truth values
    df['ma'] = x['ma'][:period_length]
    df['wma'] = x['wma'][:period_length]
    df['ewma'] = x['ewma'][:period_length]
    df['lr'] = x['lr'][:period_length]
    df['pr2'] = x['pr2'][:period_length]
    df['pr3'] = x['pr3'][:period_length]
    df['ses'] = x['ses'][:period_length]
    df['des'] = x['des'][:period_length]

    # Calculate metrics for each model
    result = []
    for model in df.columns[2:]:  # Loop through model columns (ma, ewma, etc.)
        rmse = np.sqrt(mean_squared_error(df['qty'], df[model]))  # Calculate RMSE
        r2 = r2_score(df['qty'], df[model])  # Calculate R²
        mae = mean_absolute_error(df['qty'], df[model])  # Calculate MAE
        result.append({'model': model, 'RMSE': rmse, 'MAE': mae, 'R2': r2})
    
    # Convert result to a DataFrame
    metrics_df = pd.DataFrame(result)
    
    # Select the best model (e.g., based on RMSE)
    best_model_row = metrics_df.loc[metrics_df['MAE'].idxmin()]  # Row with the lowest RMSE
    best_model = best_model_row['model']
    
    # Add the best model and metrics to the result
    return {'best_model': best_model, 'metrics': metrics_df.to_dict(orient='records')}

# Apply the metric function
data['metric'] = data.apply(lambda x: metric(x), axis=1)

# Extract the best model and metrics for each row
data['best_model'] = data['metric'].apply(lambda x: x['best_model'])
data['metrics'] = data['metric'].apply(lambda x: x['metrics'])

# Display the DataFrame
display(data[['best_model', 'metrics']])


,best_model,metrics
0,wma,"[{'model': 'ma', 'RMSE': 15.222324357824817, '..."
1,wma,"[{'model': 'ma', 'RMSE': 19.61155335879004, 'M..."
2,wma,"[{'model': 'ma', 'RMSE': 13.58527353814867, 'M..."
3,wma,"[{'model': 'ma', 'RMSE': 21.69793563237891, 'M..."
4,wma,"[{'model': 'ma', 'RMSE': 16.94143934382989, 'M..."
5,wma,"[{'model': 'ma', 'RMSE': 17.197276496411366, '..."
6,wma,"[{'model': 'ma', 'RMSE': 17.19727649641137, 'M..."
7,wma,"[{'model': 'ma', 'RMSE': 17.197276496411366, '..."
8,wma,"[{'model': 'ma', 'RMSE': 21.69793563237891, 'M..."
9,wma,"[{'model': 'ma', 'RMSE': 21.697935632378933, '..."


In [243]:
# BEST MODEL SELECTION
#def best_select(x, key):
    #return max(x, key=lambda x: x['R2'])[key]

#def best_number(x):
    #return x[x['best_model']][-1]

#data['best_model'] = data['metric'].apply(lambda x: best_select(x, 'model'))
#data['best_r2'] = data['metric'].apply(lambda x: best_select(x, 'R2'))
#data['best_value'] = data.apply(lambda x: best_number(x), axis=1)
#data['FD'] = np.ceil(data['best_value'])
#display(data)

In [244]:
data['mean_12_FD'] = data['D'].apply(lambda x: np.mean(x[-12:]))
data['std_12_FD'] = data['D'].apply(lambda x: np.std(x[-12:]))

data['ub_FD'] = data['mean_12_FD'] + 1.5 * data['std_12_FD']

data['clipped_d_FD'] = data.apply(lambda row: np.clip(row['D'][-12:], 0, row['ub_FD']).tolist(), axis=1)
def apply_best_model_forecast(row):
    best_model = row['best_model']
    
    data = row['D'][-15:] if best_model == 'wma' else row['D'][-12:]
    
    ub = row['ub_FD']
    clipped_data = np.clip(data, 0, ub).tolist()
    print(f"Clipped data for model {best_model}: {clipped_data}")
    
    if best_model == 'ma':
        ma_values = calculate_ma(clipped_data)
        forecast = ma_values[-1]
        print('ma')
    elif best_model == 'ewma':
        alpha = 0.4
        weights = np.array([(1 - alpha) ** i for i in range(len(clipped_data))][::-1])
        forecast = np.sum(weights * clipped_data) / np.sum(weights)
        print('ewma')
    elif best_model == 'wma':
        weights = [0.2, 0.3, 0.5]
        if len(clipped_data) >= len(weights):
            forecast = np.sum(np.array(clipped_data[-3:]) * weights)
        else:
            forecast = np.nan
        print('wma')
    elif best_model == 'lr':
        X = np.arange(len(clipped_data)).reshape(-1, 1)
        y = np.array(clipped_data)
        coef = np.polyfit(X.flatten(), y, 1)
        forecast = coef[0] * len(clipped_data) + coef[1]
        print('lr')
    elif best_model == 'pr2':
        X = np.arange(len(clipped_data)).reshape(-1, 1)
        y = np.array(clipped_data)
        coef = np.polyfit(X.flatten(), y, 2)
        forecast = coef[0] * (len(clipped_data) ** 2) + coef[1] * len(clipped_data) + coef[2]
        print('pr2')
    elif best_model == 'pr3':
        X = np.arange(len(clipped_data)).reshape(-1, 1)
        y = np.array(clipped_data)
        coef = np.polyfit(X.flatten(), y, 3)
        forecast = (
            coef[0] * (len(clipped_data) ** 3)
            + coef[1] * (len(clipped_data) ** 2)
            + coef[2] * len(clipped_data)
            + coef[3]
        )
    elif best_model == 'ses':
        model = SimpleExpSmoothing(clipped_data).fit(smoothing_level=0.65, optimized=False)
        forecast = model.forecast(1)[0]
        print('ses')
    elif best_model == 'des':
        model = Holt(clipped_data).fit(smoothing_level=0.65, smoothing_slope=0.45, optimized=False)
        forecast = model.forecast(1)[0]
        print('des')
    else:
        forecast = np.nan
    
    return forecast

data['FD_forecast'] = data.apply(apply_best_model_forecast, axis=1)

data['FD_final'] = data['FD_forecast'].apply(np.ceil)

display(data[['best_model', 'FD_forecast', 'FD_final']])
display(data)


Clipped data for model wma: [105.0, 110.0, 108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135.0, 140.0, 145.0, 150.0, 160.0, 165.0, 167.66898138226094]
wma
Clipped data for model wma: [210.0, 220.0, 215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260.0, 265.0, 270.0, 275.0, 285.0, 290.0, 294.234370995643]
wma
Clipped data for model wma: [55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90.0, 92.0, 95.0, 105.0, 110.0, 113.02574948440444]
wma
Clipped data for model wma: [310.0, 320.0, 315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365.0, 370.0, 375.0, 380.0, 390.0, 395.0, 400.40820205025403]
wma
Clipped data for model wma: [505.0, 510.0, 515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545.0, 550.0, 555.0, 560.0, 570.0, 575.0, 578.6677259696814]
wma
Clipped data for model wma: [155.0, 160.0, 158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190.0, 195.0, 200.0, 205.0, 215.0, 220.0, 223.66772596968153]
wma
Clipped data for model wma: [405.0, 410.0, 408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440.0, 445.0, 450.0, 455

,best_model,FD_forecast,FD_final
0,wma,165.334491,166.0
1,wma,291.117185,292.0
2,wma,110.512875,111.0
3,wma,396.704101,397.0
4,wma,575.833863,576.0
5,wma,220.833863,221.0
6,wma,470.833863,471.0
7,wma,150.833863,151.0
8,wma,346.704101,347.0
9,wma,696.704101,697.0


,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result,...,des_result,metric,best_model,metrics,mean_12_FD,std_12_FD,ub_FD,clipped_d_FD,FD_forecast,FD_final
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",135.083333,16.839727,160.342923,"[108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135...","[108.0, 111.5, 114.33333333333333, 117.0, 119....",134.695244,...,[167.0467857148429],"{'best_model': 'wma', 'metrics': [{'model': 'm...",wma,"[{'model': 'ma', 'RMSE': 15.222324357824817, '...",140.666667,18.001543,167.668981,"[115.0, 120.0, 125.0, 130.0, 128.0, 135.0, 140...",165.334491,166.0
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",256.666667,21.245915,288.535539,"[215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260...","[215.0, 222.5, 228.33333333333334, 232.5, 236....",256.544628,...,[294.9825658666633],"{'best_model': 'wma', 'metrics': [{'model': 'm...",wma,"[{'model': 'ma', 'RMSE': 19.61155335879004, 'M...",263.750000,20.322914,294.234371,"[230.0, 240.0, 245.0, 250.0, 255.0, 260.0, 265...",291.117185,292.0
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",84.416667,14.952471,106.845373,"[58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90....","[58.0, 61.5, 64.33333333333333, 67.0, 69.6, 72...",84.153781,...,[111.53189810212976],"{'best_model': 'wma', 'metrics': [{'model': 'm...",wma,"[{'model': 'ma', 'RMSE': 13.58527353814867, 'M...",89.583333,15.628277,113.025749,"[65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90.0, 92....",110.512875,111.0
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",359.583333,23.314725,394.555420,"[315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365...","[315.0, 322.5, 328.3333333333333, 332.5, 336.0...",359.546285,...,[401.0374880961348],"{'best_model': 'wma', 'metrics': [{'model': 'm...",wma,"[{'model': 'ma', 'RMSE': 21.69793563237891, 'M...",367.083333,22.216579,400.408202,"[330.0, 340.0, 345.0, 350.0, 360.0, 365.0, 370...",396.704101,397.0
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",543.333333,18.521759,571.115972,"[515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545...","[515.0, 517.5, 520.0, 522.5, 525.0, 527.5, 530...",543.009664,...,[577.7852721686249],"{'best_model': 'wma', 'metrics': [{'model': 'm...",wma,"[{'model': 'ma', 'RMSE': 16.94143934382989, 'M...",549.166667,19.667373,578.667726,"[520.0, 525.0, 530.0, 535.0, 540.0, 545.0, 550...",575.833863,576.0
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",188.166667,18.783119,216.341345,"[158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190...","[158.0, 161.5, 164.33333333333334, 167.0, 169....",187.861779,...,[222.98920400819821],"{'best_model': 'wma', 'metrics': [{'model': 'm...",wma,"[{'model': 'ma', 'RMSE': 17.197276496411366, '...",194.166667,19.667373,223.667726,"[165.0, 170.0, 175.0, 180.0, 185.0, 190.0, 195...",220.833863,221.0
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",438.166667,18.783119,466.341345,"[408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440...","[408.0, 411.5, 414.3333333333333, 417.0, 419.6...",437.861779,...,[472.9892040081983],"{'best_model': 'wma', 'metrics': [{'model': 'm...",wma,"[{'model': 'ma', 'RMSE': 17.19727649641137, 'M...",444.166667,19.667373,473.667726,"[415.0, 420.0, 425.0, 430.0, 435.0, 440.0, 445...",470.833863,471.0
7,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",118.166667,18.783119,146.341345,"[88.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0...","[88.0, 91.5, 94.33333333333333, 97.0, 99.6, 10...",117.861779,...,[152.98920400819824],"{'best_model': 'wma', 'metrics': [{'model': 'm...",wma,"[{'model': 'ma', 'RMSE': 17.197276496411366, '...",124.166667,19.667373,153.667726,"[95.0, 100.0, 105.0, 110.0, 115.0, 120.0, 125....",150.833863,151.0
8,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",309.583333,23.314725,344.555420,"[265.0, 280.0, 290.0, 295.0, 300.0, 310.0, 315...","[265.0, 272.5, 278.3333333333333, 282.5, 286.0...",309.546285,...,[351.03748809613484],"{'best_model': 'w